In [1]:
%reload_kedro

2023-10-08 10:42:41,950 - kedro.extras.extensions.ipython - INFO - No path argument was provided. Using: C:\Users\Dushku\PycharmProjects\monash-datahack-2023
2023-10-08 10:42:42,070 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2023-10-08 10:42:42,088 - kedro.framework.hooks.manager - INFO - Registered hooks from 1 installed plugin(s): kedro-telemetry-0.2.5
2023-10-08 10:42:42,138 - kedro.extras.extensions.ipython - INFO - ** Kedro project Monash datahack 2023
2023-10-08 10:42:42,139 - kedro.extras.extensions.ipython - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`


C:\Users\Dushku\PycharmProjects\monash-datahack-2023\venv\lib\site-packages\kedro\framework\context\context.py:344: UserWarning: Credentials not found in your Kedro project config.
No files found in ['C:\\Users\\Dushku\\PycharmProjects\\monash-datahack-2023\\conf\\base', 'C:\\Users\\Dushku\\PycharmProjects\\monash-datahack-2023\\conf\\local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")


In [2]:
catalog.list()

['raw_data',
 'preprocessed_data_with_type_classification',
 'preprocessed_data_with_total_classification',
 'preprocessed_data_not_recycled',
 'encoded_data_with_type_classification',
 'encoded_data_with_total_classification',
 'encoded_data_without_zero_values',
 'companies',
 'reviews',
 'shuttles',
 'data_processing.preprocessed_companies',
 'data_processing.preprocessed_shuttles',
 'model_input_table',
 'data_science.active_modelling_pipeline.regressor',
 'data_science.candidate_modelling_pipeline.regressor',
 'parameters',
 'params:data_science',
 'params:data_science.active_modelling_pipeline',
 'params:data_science.active_modelling_pipeline.model_options',
 'params:data_science.active_modelling_pipeline.model_options.test_size',
 'params:data_science.active_modelling_pipeline.model_options.random_state',
 'params:data_science.active_modelling_pipeline.model_options.features',
 'params:data_science.candidate_modelling_pipeline',
 'params:data_science.candidate_modelling_pipeline

In [3]:
df = catalog.load('preprocessed_data_with_type_classification')

2023-10-08 10:42:42,163 - kedro.io.data_catalog - INFO - Loading data from `preprocessed_data_with_type_classification` (CSVDataSet)...


In [4]:
df.head()

,Jurisdiction,Category,Type,Management,Fate,y,ds,Sub-stream
0,ACT,Biosolids,Biosolids,Other disposal,Disposal,20186,2021-12-31,3
1,ACT,Biosolids,Biosolids,Recycling,Recycling,0,2021-12-31,3
2,ACT,Building and demolition materials,Asphalt,Recycling,Recycling,0,2021-12-31,2
3,ACT,Building and demolition materials,Asphalt,Recycling,Recycling,0,2021-12-31,3
4,ACT,Building and demolition materials,Asphalt,Recycling,Recycling,0,2021-12-31,1


In [5]:
df.columns

Index(['Jurisdiction', 'Category', 'Type', 'Management', 'Fate', 'y', 'ds',
       'Sub-stream'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36851 entries, 0 to 36850
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Jurisdiction  36851 non-null  object
 1   Category      36851 non-null  object
 2   Type          36851 non-null  object
 3   Management    36851 non-null  object
 4   Fate          36851 non-null  object
 5   y             36851 non-null  int64 
 6   ds            36851 non-null  object
 7   Sub-stream    36851 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 2.2+ MB


In [7]:
df['y'] = df['y'].astype(int)

In [8]:
from sklearn.preprocessing import LabelEncoder
# Encoding the categorical values
# Initialize the LabelEncoder for each categorical column
label_encoders = {}

for col in df.columns:
    if df[col].dtype == 'object':  # Check if the column contains categorical data
        label_encoders[col] = LabelEncoder()
        df[col + '_encoded'] = label_encoders[col].fit_transform(df[col])
        
df['y'] = LabelEncoder().fit_transform(df['y'])

In [9]:
df.columns

Index(['Jurisdiction', 'Category', 'Type', 'Management', 'Fate', 'y', 'ds',
       'Sub-stream', 'Jurisdiction_encoded', 'Category_encoded',
       'Type_encoded', 'Management_encoded', 'Fate_encoded', 'ds_encoded'],
      dtype='object')

In [10]:
# Selecting encoded variables along with the y variable
df = df[['y', 'Jurisdiction_encoded', 'Sub-stream', 'Category_encoded', 'Type_encoded', 'Management_encoded', 'Fate_encoded', 'ds_encoded']]

In [11]:
catalog.save("encoded_data_with_type_classification", df)

2023-10-08 10:42:42,568 - kedro.io.data_catalog - INFO - Saving data to `encoded_data_with_type_classification` (CSVDataSet)...


In [12]:
df = catalog.load('preprocessed_data_with_total_classification')

2023-10-08 10:42:42,614 - kedro.io.data_catalog - INFO - Loading data from `preprocessed_data_with_total_classification` (CSVDataSet)...


In [13]:
df['y'] = df['y'].astype(int)

In [14]:
from sklearn.preprocessing import LabelEncoder
# Encoding the categorical values
# Initialize the LabelEncoder for each categorical column
label_encoders = {}

for col in df.columns:
    if df[col].dtype == 'object':  # Check if the column contains categorical data
        label_encoders[col] = LabelEncoder()
        df[col + '_encoded'] = label_encoders[col].fit_transform(df[col])
        
df['y'] = LabelEncoder().fit_transform(df['y'])

In [15]:
df.columns

Index(['Jurisdiction', 'Category', 'Type', 'Total type', 'Management', 'Fate',
       'y', 'ds', 'Sub-stream', 'Jurisdiction_encoded', 'Category_encoded',
       'Type_encoded', 'Total type_encoded', 'Management_encoded',
       'Fate_encoded', 'ds_encoded'],
      dtype='object')

In [16]:
# Selecting encoded variables along with the y variable
df = df[['y', 'Jurisdiction_encoded', 'Sub-stream', 'Total type', 'Category_encoded', 'Type_encoded', 'Management_encoded', 'Fate_encoded', 'ds_encoded']]

In [17]:
catalog.save("encoded_data_with_total_classification", df)

2023-10-08 10:42:42,693 - kedro.io.data_catalog - INFO - Saving data to `encoded_data_with_total_classification` (CSVDataSet)...


In [18]:
df = catalog.load('preprocessed_data_not_recycled')

2023-10-08 10:42:42,708 - kedro.io.data_catalog - INFO - Loading data from `preprocessed_data_not_recycled` (CSVDataSet)...


In [19]:
df['y'] = df['y'].astype(int)

In [20]:
from sklearn.preprocessing import LabelEncoder
# Encoding the categorical values
# Initialize the LabelEncoder for each categorical column
label_encoders = {}

for col in df.columns:
    if df[col].dtype == 'object':  # Check if the column contains categorical data
        label_encoders[col] = LabelEncoder()
        df[col + '_encoded'] = label_encoders[col].fit_transform(df[col])
        
df['y'] = LabelEncoder().fit_transform(df['y'])

In [21]:
df.columns

Index(['Jurisdiction', 'Category', 'Type', 'Classification', 'Total type',
       'Stream', 'Management', 'Fate', 'y', 'Core or non-core', 'Sub-stream',
       'Cat. order', 'Cat. no.', 'Type order', 'Type no.', 'Start date', 'ds',
       'Jurisdiction_encoded', 'Category_encoded', 'Type_encoded',
       'Classification_encoded', 'Total type_encoded', 'Stream_encoded',
       'Management_encoded', 'Fate_encoded', 'Core or non-core_encoded',
       'Cat. no._encoded', 'Type no._encoded', 'Start date_encoded',
       'ds_encoded'],
      dtype='object')

In [22]:
# Selecting encoded variables along with the y variable
df = df[['y', 'Jurisdiction_encoded', 'Sub-stream', 'Category_encoded', 'Type_encoded', 'Management_encoded', 'Fate_encoded', 'ds_encoded']]

In [23]:
df.head()

,y,Jurisdiction_encoded,Sub-stream,Category_encoded,Type_encoded,Management_encoded,Fate_encoded,ds_encoded
0,4748,0,3,1,7,2,0,14
1,4748,0,9,1,7,2,0,14
2,4748,0,3,1,68,2,0,14
3,4748,0,9,1,68,2,0,14
4,4748,0,3,1,68,2,0,14


In [24]:
catalog.save("preprocessed_data_not_recycled", df)

2023-10-08 10:42:42,896 - kedro.io.data_catalog - INFO - Saving data to `preprocessed_data_not_recycled` (CSVDataSet)...
